In [1]:
#установка
#pip install pyxll
#pip install --upgrade pip
#pip install pyxll
#pip install pyxll-jupyter
#pip install openpyxl

In [17]:
# импорт библиотек
import yadisk
import os
from tqdm import tqdm
import openpyxl
from openpyxl import Workbook
import sys
import datetime
import pandas as pd

import requests

In [3]:
#полученные от яндекса 
app_id = 'здесь app_id'
secret_id = 'здесь secret_id'

#token='здесь token'

In [4]:
y = yadisk.YaDisk(app_id, secret_id)
url = y.get_code_url()
print("Go to the following url: %s" % url)
code = input("Enter the confirmation code: ")
try:
 response = y.get_token(code)
except yadisk.exceptions.BadRequestError:
 print("Bad code")
 sys.exit(1)
y.token = response.access_token
if y.check_token():
 print("Sucessfully received token!")
else:
 print("Something went wrong. Not sure how though...")


Go to the following url: https://oauth.yandex.ru/authorize?response_type=code&client_id=590e92659bcf4cc0ac1915c4a65c4e3f&force_confirm=yes
Enter the confirmation code: 7631597
Sucessfully received token!


In [5]:
# проверяем токен
y.check_token()

True

In [6]:
# создаем список файлов для загрузки
list_of_files = []
for el in list(y.listdir('Test_first')):
    if el['path'].endswith('.xlsx'):
        list_of_files.append(el['path'])

In [7]:
# создаем папку загрузки
load_path = 'C:/Users/User/my_env/test'
if not os.path.exists(load_path):
     os.mkdir(load_path)
os.chdir(load_path)

In [8]:
# загружаем файлы с яндекс диска
for file in tqdm(list_of_files):
     y.download(file.split(':')[1], file.split('/')[-1])

100%|████████████████████████████████████████████████████████████████████████████████| 162/162 [02:30<00:00,  1.07it/s]


In [9]:
#список файлов на локальном компьютере, с которыми будем работать
local_list=os.listdir('C:/Users/User/my_env/test')
  

In [ ]:
#файл с полями _субъектРФ, _регион, _полигон_координатами,точечные_координаты
#файл с _субъекатмиРФ, _регион. В названии регионов убираем крайние пробелы
region_dv=pd.read_excel('C:/Users/User/my_env/test/Region_subgect.xlsx')
region_dv.columns=[ 'subject', 'region']
region_dv['region']=region_dv['region'].str.strip()

#файл с регионами и координатами, так же убираем крайние пробелы
region_coor=pd.read_csv('regions_coords.csv')
region_coor['region']=region_coor['region'].str.strip()


region_coor_all=region_dv.merge(region_coor, on='region', how='left')

In [10]:
#1.Функция, которая собирает отчет по ЗНО 
#Список фалов, содержащих zno 
zno_list=[]
subs='Злокачественные_новообразования_в_РФ'
for i in (os.listdir('C:/Users/User/my_env/test')):
    #в папке есть файлы с нмоерами 005, 006, 007, 069 - они не нужны для анализа
    if (subs in i and '00' not in i): 
        if (subs in i and '65' not in i):
            zno_list.append(i) 


#сборная таблица
test_zno=[]
test_all=[]

cals_test=['region','abs','rude_100','standart_100','error_100',
          'М_абсолютное число','М_на 100 тыс.грубый','М_на 100 тыс.стандарт','М_на 100 тыс.ошибка',
          'Ж_абсолютное число','Ж_на 100 тыс.грубый','Ж_на 100 тыс.стандарт','Ж_на 100 тыс.ошибка']
cals_test5=['region','abs','rude_100','standart_100','error_100', 'gender']

flag=0 #флаг для первой сборки    

for k in zno_list:
    test12=pd.read_excel(k, header=[1, 2, 3, 4, 5, 6])
    
    #проверка на количество столбцов 5 или 13 
    if len(test12.columns)==5:
        test12=pd.read_excel(k, header=[1, 2, 3, 4, 5]) #в "гендерных заболеваниях" заголовок на 1 меньше
        test12['gender']='гендерное заболевание'
        test12.columns=cals_test5
        test_zno=test12
    else:    
        test12.columns=cals_test
        t_all=test12.loc[:,['region','abs','rude_100','standart_100','error_100']]
        t_all['gender']='всё население'
        t_man=test12.loc[:,['region','М_абсолютное число','М_на 100 тыс.грубый','М_на 100 тыс.стандарт','М_на 100 тыс.ошибка']]
        t_man['gender']='мужчины'
        t_man.columns=cals_test5
        t_woman=test12.loc[:,['region','Ж_абсолютное число','Ж_на 100 тыс.грубый','Ж_на 100 тыс.стандарт','Ж_на 100 тыс.ошибка']]
        t_woman['gender']='женщины'
        t_woman.columns=cals_test5
        test_zno=pd.concat([t_all,t_man, t_woman])
    
    #добавим столбцы с информацией из заголовков
    test_ex=openpyxl.reader.excel.load_workbook(filename=k)
    table_name=test_ex.active['A1'].value
    table_zc=test_ex.active['A2'].value.lower().split()[0]
    
    #проверка на корректность объединения ячеек в исследуемом файле
    if test_ex.active['B3'].value != None:
        table_year=test_ex.active['B3'].value
    else:
        table_year=test_ex.active['A3'].value.split(': ')[1]
   
    if test_ex.active['B4'].value != None:
            table_local=test_ex.active['B4'].value
    else:
            table_local=test_ex.active['A4'].value.split(': ')[1]
                     
    
    
    #добавляем в сборную таблицу дополнительную информацию
    test_zno['ind']=table_zc
    test_zno['year']=table_year
    test_zno['loc']=table_local
    test_zno['table']=table_name
    test_zno['bzz']='ЗНО'
    
    #проверяем - первый раз сборка или нет через flag
    if flag==0:
        test_all=test_zno
        flag=1
    else:
        test_all=pd.concat([test_all, test_zno], ignore_index=True)
        

#убираем лишние пробелы в названии регионов, добавляем к таблице колонки с координатами, 
#выводим название субъектов  в отдельную колонку
test_all['region']=test_all['region'].str.strip()
test_all=region_coor_all.merge(test_all, on='region', how='left')

#приводим итоговую таблицу к таблице в Паспортизации
test_all=test_all[['region', 'subject', 'gender', 'abs', 'rude_100',
                   'standart_100','error_100', 'ind', 
                   'year','loc','table','bzz', 'polygon_coords']]


test_all.to_excel('ONKO_zno.xlsx')
#первый очтет готов!

In [11]:
#2. Функция, которая собирает отчет по онко58-82
# Список файлов, содержащих
onko_rl=[]
subs='Состояние_онко_помощи_в_РФ'
for i in (os.listdir('C:/Users/User/my_env/test')):
    for k in range(58,86):
        if (subs in i and str(k) in i):
            onko_rl.append(i) 



onko_all=[]
flag=0
onko_col=['region', 'zno_end', 'zno_first', 
          'zno_cont', 'zno_perc', 'met_sur', 
          'met_rad', 'met_med', 'met_com', 'met_h_r']

for k in onko_rl:
    test12=pd.read_excel(k, header=[1, 2])
    test12.columns=onko_col 
               
    #добавим столбцы с информацией из заголовков
    test_ex=openpyxl.reader.excel.load_workbook(filename=k)
    table_name='Т'+test_ex.active['A1'].value.split(' Т')[1]
    table_local=test_ex.active['A1'].value.split('\n')[2].replace(table_name,'').strip()
          
    #добавляем в сборную таблицу дополнительную информацию
    test12['ind']='СВЕДЕНИЯ О ЛЕЧЕНИИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВАНИЙ (ЗНО), ВПЕРВЫЕ ЗАРЕГИСТРИРОВАННЫХ В 2021 Г., ПОДЛЕЖАЩИХ РАДИКАЛЬНОМУ ЛЕЧЕНИЮ'
    test12['year']='2021'
    test12['loc']=table_local
    test12['table']=table_name
    test12['bzz']='СОП'
    
    #проверяем - первый раз сборка или нет через flag
    if flag==0:
        onko_all=test12
        flag=1
    else:
        onko_all=pd.concat([onko_all, test12], ignore_index=True)
    
#убираем лишние пробелы в названии регионов, добавляем к таблице колонки с координатами, 
#выводим название субъектов  в отдельную колонку
onko_all['region']=onko_all['region'].str.strip()
onko_all=region_coor_all.merge(onko_all, on='region', how='left')

#приводим итоговую таблицу к таблице в Паспортизации
onko_all=onko_all[['region', 'subject', 'zno_end', 'zno_first',
                  'zno_cont', 'zno_perc', 'met_sur', 'met_rad', 
                  'met_med', 'met_com', 'met_h_r', 'ind', 'year', 'loc', 
                  'table', 'bzz', 'polygon_coords']]

#выводим итоговую таблицу

onko_all.to_excel('ONKO_stat.xlsx')

#Второй отчет - готов!

In [12]:
#3. Функция, которая собирает отчет по онко24-51, sheet1

# Список файлов с первым типом заголовка + 32
er1=['24', '25', '26',
    '27','28', '29', 
    '30', '31', '32',
    '39','40', '41',
    '42', '45','46',
     '47', '49','50',
     '51', '52','53']

onko_rl=[]
subs='Состояние_онко_помощи_в_РФ'
for i in (os.listdir('C:/Users/User/my_env/test')):
    for k in er1:
        if (subs in i and str(k) in i):
            onko_rl.append(i) 

onko_all=[]
flag=0
onko_col=['region', 'onko_first', 'activ_perc', 
          'abs_year', '1000_year', 
          '5_abs', '5_perc',
          'index_cont', 'leth']

for k in onko_rl:
    test12=pd.read_excel(k, sheet_name=0, header=[1, 2])
    test12.columns=onko_col 
               
    #добавим столбцы с информацией из заголовков
    test_ex=openpyxl.reader.excel.load_workbook(filename=k)
    test_ex.active=0
    
    #в таблице 32 забыли написать на первом листе, что это таблица 32
    if k=='2021_Таблица_032_Состояние_онко_помощи_в_РФ.xlsx':
        table_name='Таблица 32'
        table_local='Ободочная кишка (С18)'
    else:
        table_name='Т'+test_ex.active['A1'].value.split(' Т')[1].strip()
        table_local=test_ex.active['A1'].value.split('.\n')[1].replace(table_name,'').strip()

    #добавляем в сборную таблицу дополнительную информацию
    test12['ind']='СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.'
    test12['year']='2021'
    test12['loc']=table_local
    test12['table']=table_name
    test12['bzz']='СОП'
    
    #проверяем - первый раз сборка или нет через flag
    if flag==0:
        onko_all=test12
        flag=1
    else:
        onko_all=pd.concat([onko_all, test12], ignore_index=True)

#файлы со вторым типом информации
er2=['33', '34', '35',
    '36','37', '38', '43',
    '44', '48']

onko_rl=[]
for i in (os.listdir('C:/Users/User/my_env/test')):
    for k in er2:
        if (subs in i and str(k) in i):
            onko_rl.append(i)
            
for k in onko_rl:
    test12=pd.read_excel(k, sheet_name=0, header=[1, 2])
    test12.columns=onko_col 
               
    #добавим столбцы с информацией из заголовков
    test_ex=openpyxl.reader.excel.load_workbook(filename=k)
    test_ex.active=0
    table_local=test_ex.active['A1'].value.split('\n')[1].strip()
    table_name='Т'+test_ex.active['A1'].value.split(' Т')[1].replace(table_local, '').strip()

    #добавляем в сборную таблицу дополнительную информацию
    test12['ind']='СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.'
    test12['year']='2021'
    test12['loc']=table_local
    test12['table']=table_name
    test12['bzz']='СОП'
    
    onko_all=pd.concat([onko_all, test12], ignore_index=True)

#убираем лишние пробелы в названии регионов, добавляем к таблице колонки с координатами, 
#выводим название субъектов  в отдельную колонку
onko_all['region']=onko_all['region'].str.strip()
onko_all=region_coor_all.merge(onko_all, on='region', how='left')

#приводим итоговую таблицу к таблице в Паспортизации
onko_all=onko_all[['region', 'subject', 'onko_first', 
                   'activ_perc', 'abs_year', '1000_year', 
                   '5_abs', '5_perc', 'index_cont',
                   'leth', 'ind', 'year', 'loc',
                   'table', 'bzz', 'polygon_coords']]


onko_all.to_excel('ONKO_ cont_1.xlsx')     

#третий отчет готов

In [13]:
#4. функция, которая собирает отчет по онко 24-51, sheet2

onko_rl=[]
table_name_df=[]
subs='Состояние_онко_помощи_в_РФ'
for i in (os.listdir('C:/Users/User/my_env/test')):
    for k in range(24, 52):
        if (subs in i and str(k) in i):
            # переменная s позволяет указать номер таблицы(файла), т.к. в самих фалах беспорядок на "Продолжение таблицы"
            onko_rl.append(i)
            s=i.replace('2021_', '')
            s=s.replace('_Состояние_онко_помощи_в_РФ.xlsx', '')
            s=s.replace('_',' ')
            table_name_df.append(s)
            
onko_all=[]
flag=0
dn=0 #флаг для указателя из какого файла взяли информацию для table_name

onko_col=['region', 'reg_all', 'diagnos', 
          'stad_one', 'stad_two', 
          'stad_three', 'stad_four',
          'stad_none', 'leth_first_year']

for k in onko_rl:
    test12=pd.read_excel(k, sheet_name=1, header=[1, 2, 3])
    test12.columns=onko_col 
    
    #добавим столбцы с информацией из заголовков
    test_ex=openpyxl.reader.excel.load_workbook(filename=k)
    test_ex.active=1
    table_name=table_name_df[dn]
    dn=dn+1
    table_local=(test_ex.active['A1'].value.split('\n')[1]).split(')',1)[0].strip()+')'

    #добавляем в сборную таблицу дополнительную информацию
    test12['ind']='ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВАНИЙ, ВЫЯВЛЕННЫХ В 2021 Г.'
    test12['year']='2021'
    test12['loc']=table_local
    test12['table']=table_name
    test12['bzz']='СОП'
    
    #проверяем - первый раз сборка или нет через flag
    if flag==0:
        onko_all=test12
        flag=1
    else:
        onko_all=pd.concat([onko_all, test12], ignore_index=True)

#убираем лишние пробелы в названии регионов, добавляем к таблице колонки с координатами, 
#выводим название субъектов  в отдельную колонку
onko_all['region']=onko_all['region'].str.strip()
onko_all=region_coor_all.merge(onko_all, on='region', how='left')


#приводим итоговую таблицу к таблице в Паспортизации
onko_all=onko_all[['region', 'subject', 'reg_all', 
                   'diagnos','stad_one','stad_two',
                   'stad_three','stad_four','stad_none',
                   'leth_first_year','ind','year',
                   'loc','table','bzz','polygon_coords']]

        
onko_all.to_excel('ONKO_ cont_2.xlsx')  
#четвертый отчет готов